#### Included:
Gene expression change (Baseline)
Gene expression change + Kegg pathways**
ssGSEA change
Gene expression change + ssGSEA

In [1]:
import pandas as pd
import thesis_functions as tf
from aerial import model, rule_extraction, rule_quality, discretization
import numpy as np
from mlxtend.frequent_patterns import fpgrowth, association_rules


### Baseline

In [3]:
# Load expression data
expr_matrix_raw = pd.read_csv('Data/gene_expr_matrix_VLCD_LCD_2timepoints.csv', index_col=0) 
metadata = pd.read_csv('Data/metadata_filtered_2timepoints_VLCD_LCD.csv', index_col=0)
# Rename expr_matrix_raw columns using super_id from metadata
expr_matrix_raw.columns = metadata.loc[expr_matrix_raw.columns, 'super_id'].astype(str).values

# Subset only one treatment
super_ids_VLCD = metadata.loc[metadata['treatment'] == 'very-low-calorie diet', 'super_id']
expr_matrix_raw = expr_matrix_raw.loc[:, expr_matrix_raw.columns.isin(super_ids_VLCD)]
expr_matrix_raw.head()

# significant subset only
sig_genenames = pd.read_csv('Data/sig_genenames_VLCD_LCD.csv')
expr_matrix_raw_sig = expr_matrix_raw.loc[expr_matrix_raw.index.intersection(sig_genenames['x'])]
print(expr_matrix_raw_sig.shape) #250 genes, 50 samples

(250, 50)


In [21]:
kegg_df = pd.read_csv('kegg_pathway_matrix_229.csv')
kegg_df = kegg_df.set_index('SYMBOL', drop=True)
kegg_df.head()

,hsa00010 (Glycolysis / Gluconeogenesis),hsa00030 (Pentose phosphate pathway),hsa00040 (Pentose and glucuronate interconversions),hsa00051 (Fructose and mannose metabolism),hsa00053 (Ascorbate and aldarate metabolism),hsa00061 (Fatty acid biosynthesis),hsa00062 (Fatty acid elongation),hsa00071 (Fatty acid degradation),hsa00100 (Steroid biosynthesis),hsa00120 (Primary bile acid biosynthesis),...,hsa05330 (Allograft rejection),hsa05332 (Graft-versus-host disease),hsa05340 (Primary immunodeficiency),hsa05410 (Hypertrophic cardiomyopathy),hsa05412 (Arrhythmogenic right ventricular cardiomyopathy),hsa05414 (Dilated cardiomyopathy),hsa05415 (Diabetic cardiomyopathy),hsa05416 (Viral myocarditis),hsa05417 (Lipid and atherosclerosis),hsa05418 (Fluid shear stress and atherosclerosis)
SYMBOL,,,,,,,,,,,,,,,,,,,,,
FADS2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FASN,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SCD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TNFRSF25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LOXL2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
expr_matrix_raw_sig = expr_matrix_raw.loc[expr_matrix_raw.index.intersection(kegg_df.index)]
expr_matrix_raw_sig_logfc = tf.GetFoldChangeDf(expr_matrix_raw_sig, subjects_as_rows=True)
baseline_df = tf.MakeOneHotDf(expr_matrix_raw_sig_logfc, threshold=0.1)
print(baseline_df.shape)
baseline_df.head()


(24, 458)


,AACS_downregulated,AACS_upregulated,AADAC_downregulated,AADAC_upregulated,ABCC6P2_downregulated,ABCC6P2_upregulated,ABCC6_downregulated,ABCC6_upregulated,ABCD2_downregulated,ABCD2_upregulated,...,TREM2_downregulated,TREM2_upregulated,TYROBP_downregulated,TYROBP_upregulated,VLDLR-AS1_downregulated,VLDLR-AS1_upregulated,VLDLR_downregulated,VLDLR_upregulated,XAGE-4_downregulated,XAGE-4_upregulated
Subject,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,1,0,1,0,1,0,...,0,1,0,1,1,0,1,0,1,0
2,1,0,0,1,1,0,0,0,1,0,...,0,1,0,1,1,0,1,0,1,0
3,1,0,0,1,1,0,1,0,1,0,...,0,1,0,1,1,0,1,0,1,0
4,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,0,0,0,1,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1


In [13]:
min_sup = 0.7
min_conf = 0.8
ae_results, ae_rules = tf.RunPyAerialnTimes(baseline_df, ant_sim=min_sup, cons_sim=min_conf, save_rules=True, target_classes=False,
                                             features_of_interest_onehot=True, filter_thresholds=True, batch_size=5, n_epochs=5)

number of rules before filtering 325920
number of rules after filtering 115
       Exec Time (s)  Nr Rules  avg Confidence  avg Support  avg Coverage  \
Run 1      22.200153     115.0           0.923         0.72           1.0   

       avg Zhangs Metric  
Run 1              0.533  
number of rules before filtering 109395
number of rules after filtering 251
       Exec Time (s)  Nr Rules  avg Confidence  avg Support  avg Coverage  \
Run 1      22.200153     115.0           0.923         0.72           1.0   
Run 2      17.228753     251.0           0.907         0.73           1.0   

       avg Zhangs Metric  
Run 1              0.533  
Run 2              0.288  
number of rules before filtering 2700696
number of rules after filtering 456
       Exec Time (s)  Nr Rules  avg Confidence  avg Support  avg Coverage  \
Run 1      22.200153     115.0           0.923         0.72           1.0   
Run 2      17.228753     251.0           0.907         0.73           1.0   
Run 3      44.8753

In [14]:
ae_rules

[{'antecedents': ['ACP5_upregulated__1'],
  'consequent': 'ALDOC_downregulated__1',
  'support': 0.708,
  'confidence': 0.944,
  'zhangs_metric': 0.294,
  'rule_coverage': 0.75},
 {'antecedents': ['ACP5_upregulated__1'],
  'consequent': 'C6_upregulated__1',
  'support': 0.708,
  'confidence': 0.944,
  'zhangs_metric': 0.118,
  'rule_coverage': 0.75},
 {'antecedents': ['ACP5_upregulated__1'],
  'consequent': 'MSR1_upregulated__1',
  'support': 0.708,
  'confidence': 0.944,
  'zhangs_metric': 1.0,
  'rule_coverage': 0.75},
 {'antecedents': ['ACP5_upregulated__1'],
  'consequent': 'STC1_downregulated__1',
  'support': 0.708,
  'confidence': 0.944,
  'zhangs_metric': 0.824,
  'rule_coverage': 0.75},
 {'antecedents': ['ALCAM_upregulated__1'],
  'consequent': 'ALDOC_downregulated__1',
  'support': 0.708,
  'confidence': 0.944,
  'zhangs_metric': 0.294,
  'rule_coverage': 0.75},
 {'antecedents': ['ALCAM_upregulated__1'],
  'consequent': 'C6_upregulated__1',
  'support': 0.708,
  'confidence':

In [16]:
metrics_list = ['confidence', 'support', 'zhangs_metric', 'lift']

# Perform FP growth with filter and save metrics
itemsets = fpgrowth(baseline_df, min_support=min_sup, use_colnames=True, max_len=3)
if itemsets.empty:
    print("No frequent itemsets found with the given min_support.")
    fp_rules = pd.DataFrame()
else:
    fp_rules = association_rules(itemsets, metric="confidence", min_threshold=min_conf, return_metrics= metrics_list)

fp_rules
print(len(fp_rules.index))

1343


/Users/louisebuijs/Documents/DataScience Master/Thesis/.venv/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [18]:
metrics_to_average = ['support', 'confidence', 'zhangs_metric']
results_ae_df = pd.DataFrame(ae_rules)
print(results_ae_df[metrics_to_average].mean())
print(f"avg nr of rules AE:{ae_results['Nr Rules'].mean()}")
print(fp_rules.iloc[:,2:-1].mean())


support          0.723861
confidence       0.929916
zhangs_metric    0.471198
dtype: float64
avg nr of rules AE:397.4
confidence       0.908873
support          0.722977
zhangs_metric    0.544475
dtype: float64


#### Gene | kegg


In [6]:
# Load kegg data (it has a slightly different number of genes because when it was created I was using a significant gene set 
# that was later changed due to different method for getting sig genes)
kegg_df = pd.read_csv('kegg_pathway_matrix_229.csv')
kegg_df = kegg_df.set_index('SYMBOL', drop=True)
kegg_df.head()

# take first 5 genes (index is gene names) 
genes_sample = kegg_df.index[5:10]
print (genes_sample)

# Find all columns (hsa terms) attributed to any of those genes
relevant_hsa_terms = kegg_df.loc[genes_sample].sum(axis=0)
relevant_hsa_terms = relevant_hsa_terms[relevant_hsa_terms > 0].index

# Filter kegg matrix to relevant genes and terms
filtered_kegg = kegg_df.loc[genes_sample, relevant_hsa_terms]

# Find those genes in expression matrix
expr_matrix_raw_tiny = expr_matrix_raw.loc[expr_matrix_raw.index.intersection(genes_sample)]
tiny_fc_df = tf.GetFoldChangeDf(expr_matrix_raw_tiny, subjects_as_rows=True)
tiny_discr_df = tf.discretize_features_by_abs_threshold(tiny_fc_df, threshold=0.1)

# Horizontal expansion: gene|hsaterm
expanded_columns = []
expanded_data = []

for gene in filtered_kegg.index:
    for hsa_term in filtered_kegg.columns:
        if filtered_kegg.loc[gene, hsa_term] == 1:
            col_name = f"{gene}|{hsa_term}"
            expanded_columns.append(col_name)
            expanded_data.append(tiny_discr_df[gene].values)

expanded_df = pd.DataFrame(
    data=np.array(expanded_data).T,  # shape: (num_samples, num_features)
    index=tiny_discr_df.index,       # sample IDs
    columns=expanded_columns         # GENE|hsaXXXX
)
# Final output
print(expanded_df.head())

Index(['SRPX', 'FADS1', 'ALDOC', 'KLB', 'GPX3'], dtype='object', name='SYMBOL')
  FADS1|hsa01040 (Biosynthesis of unsaturated fatty acids)  \
1                                           NoChange         
2                                               Down         
3                                               Down         
4                                               Down         
5                                                 Up         

  FADS1|hsa01100 (Metabolic pathways) FADS1|hsa01212 (Fatty acid metabolism)  \
1                            NoChange                               NoChange   
2                                Down                                   Down   
3                                Down                                   Down   
4                                Down                                   Down   
5                                  Up                                     Up   

  ALDOC|hsa00010 (Glycolysis / Gluconeogenesis)  \
1              

In [12]:
expanded_df.shape

(24, 17)

The problem with this approach is that each rule between genes is basically captured a lot of times because the columns are the same for each gene\term combination. See code below to see the duplicate rules.

In [9]:
min_sup = 0.7
min_conf = 0.8
aerial_tiny_df_discr_results, aerial_tiny_df_discr_rules = tf.RunPyAerialnTimes(tiny_discr_df, ant_sim=min_sup, cons_sim=min_conf, features_of_interest_onehot=False, target_classes=False, filter_thresholds=True)
print(aerial_tiny_df_discr_rules)

number of rules before filtering 10
number of rules after filtering 3
       Exec Time (s)  Nr Rules  avg Confidence  avg Support  avg Coverage  \
Run 1       9.286072       3.0           0.935         0.78         0.958   

       avg Zhangs Metric  
Run 1              0.589  
number of rules before filtering 5
number of rules after filtering 2
       Exec Time (s)  Nr Rules  avg Confidence  avg Support  avg Coverage  \
Run 1       9.286072       3.0           0.935         0.78         0.958   
Run 3       9.106440       2.0           0.950         0.77         0.958   

       avg Zhangs Metric  
Run 1              0.589  
Run 3              0.384  
number of rules before filtering 1
number of rules after filtering 1
       Exec Time (s)  Nr Rules  avg Confidence  avg Support  avg Coverage  \
Run 1       9.286072       3.0           0.935         0.78         0.958   
Run 3       9.106440       2.0           0.950         0.77         0.958   
Run 4       9.071460       1.0         

In [17]:
aerial_gene_hsa_results,  aerial_gene_hsa_rules = tf.RunPyAerialnTimes(expanded_df, ant_sim=min_sup, cons_sim=min_conf, features_of_interest_onehot=False, target_classes=False, filter_thresholds=True)
print(aerial_gene_hsa_results)
print(len(aerial_gene_hsa_rules))

metrics_to_average = ['support', 'confidence', 'zhangs_metric', 'lift']
results_ae_df = pd.DataFrame(aerial_gene_hsa_rules)
print(results_ae_df[metrics_to_average].mean())
print(f"avg nr of rules AE:{aerial_gene_hsa_results['Nr Rules'].mean()}")


number of rules before filtering 336
number of rules after filtering 317
       Exec Time (s)  Nr Rules  avg Confidence  avg Support  avg Coverage  \
Run 1       9.192059     317.0            0.99         0.82         0.958   

       avg Zhangs Metric  
Run 1              0.728  
number of rules before filtering 510
number of rules after filtering 317
       Exec Time (s)  Nr Rules  avg Confidence  avg Support  avg Coverage  \
Run 1       9.192059     317.0           0.990         0.82         0.958   
Run 2       8.351701     317.0           0.988         0.80         0.958   

       avg Zhangs Metric  
Run 1              0.728  
Run 2              0.688  
number of rules before filtering 252
number of rules after filtering 203
       Exec Time (s)  Nr Rules  avg Confidence  avg Support  avg Coverage  \
Run 1       9.192059     317.0           0.990         0.82         0.958   
Run 2       8.351701     317.0           0.988         0.80         0.958   
Run 3       8.408106     203